## 🤝 Step 6: Critical Thinking, Explainability, Ethical AI, and Bias Auditing

---
This step evaluates the recommender system beyond accuracy by examining explainability, ethical considerations, and bias risks. Because the solution is a content-based recommender with no user profiles or prediction labels, traditional supervised explainability and fairness techniques are not directly applicable. Instead, this step applies recommender-appropriate methods to explain why recommendations are generated, audit content exposure bias, document mitigation strategies, and transparently disclose system limitations. The goal is to ensure the system is interpretable, auditable, and responsible under real-world, data-limited conditions.

In [ ]:
# =============================================================================
# STEP 6: CRITICAL THINKING, EXPLAINABILITY, ETHICAL AI, AND BIAS AUDITING
# =============================================================================

"""
Review the recommender system from an explainability, ethics, and bias
perspective to ensure the solution is responsible, transparent, and
aligned with real-world constraints.

Because this project uses a content-based recommender without user data
or prediction labels, explainability and fairness are handled differently
from traditional supervised machine learning models.

This step focuses on:
- Explaining why recommendations are generated
- Checking for bias in content exposure (not user demographics)
- Identifying risks and mitigation strategies
- Clearly documenting system limitations

Industry tools like SHAP and LIME are referenced conceptually, but are
replaced with recommender-appropriate techniques that match the system design.
"""

# ---------------------------------------------------------------------------
# 6a. Explainability – Term Overlap Analysis (TF-IDF Inspired)
# ---------------------------------------------------------------------------
# Purpose: Explain *why* two titles are considered similar using shared
# high-importance terms instead of black-box scores.

# Conceptual reference: SHAP / LIME
# Practical replacement: Inspect which important terms overlap between titles

def explain_term_overlap(query_idx, rec_indices, tfidf_matrix, feature_names, top_n=10):
    """
    Identify the most important shared terms that explain why titles
    are considered similar.

    Purpose:
    Provide a clear, human-readable explanation for recommendations
    by highlighting overlapping content terms.
    """
    query_vector = tfidf_matrix[query_idx].toarray().flatten()          # Extract TF-IDF values so we can see which terms matter most for the query title.
    explanations = []                                                   # Store explanations so results can be reviewed in table form.

    for ridx in rec_indices:
        rec_vector = tfidf_matrix[ridx].toarray().flatten()             # Extract TF-IDF values for the recommended title for fair comparison.
        overlap = query_vector * rec_vector                             # Multiply vectors to isolate shared important terms.
        top_terms_idx = overlap.argsort()[::-1][:top_n]                 # Rank shared terms by contribution strength so explanations stay concise.
        top_terms = [feature_names[i] for i in top_terms_idx if overlap[i] > 0]  # Keep only terms that actually contribute.

        explanations.append({
            "Recommended Index": ridx,
            "Top Overlapping Terms": ", ".join(top_terms[:top_n])       # Present terms in readable format for non-technical users.
        })

    return pd.DataFrame(explanations)                                   # Return as a DataFrame for clarity and reporting.


query_idx = 0                                                           # Use one example title to demonstrate explainability clearly.
rec_indices = get_top_k_indices(query_idx, cosine_sim_matrix, top_k=5)  # Retrieve a small Top-K list to keep explanations focused.

term_overlap_df = explain_term_overlap(
    query_idx,
    rec_indices,
    tfidf_matrix,
    tfidf.get_feature_names_out(),
    top_n=8
)

print("Explainability – Term Overlap Analysis")
display(term_overlap_df)
print(
    f"INTERPRETATION:\nThe shared terms shown above explain why the selected \n"
    f"title '{df.loc[query_idx, 'title']}' is matched with these recommendations. \n"
    f"This mirrors the intent of SHAP or LIME by making similarity decisions \n"
    f"transparent and understandable. \n"
)
print("\n")


# ---------------------------------------------------------------------------
# 6b. Explainability – Feature Contribution Discussion (Weighted Model)
# ---------------------------------------------------------------------------
# Purpose: Show how much each content signal contributes so model behavior
# can be explained in simple business terms.

# Conceptual reference: Feature importance attribution

feature_contributions = pd.DataFrame({
    "Feature": ["Genre", "Description"],
    "Weight": [best_genre_weight, 1 - best_genre_weight]
})                                                               # Display weights to make trade-offs explicit.

print("Feature Contribution Discussion (Weighted Model)")
display(feature_contributions)
print(
    f"INTERPRETATION:\nThe model relies more on genre ({best_genre_weight:.2f}) than descriptions ({1 - best_genre_weight:.2f}). \n"
    f"This improves interpretability and allows alignment with business preferences. \n"
)
print("\n")


# ---------------------------------------------------------------------------
# 6c. Bias Auditing – Genre Exposure Analysis
# ---------------------------------------------------------------------------
# Purpose: Check whether recommendations disproportionately favor
# popular genres compared to the full catalog.

# Bias focus: Content exposure, not user demographics


# Clean GENRE field to remove NaN or blank (" ") genre, found this bug during the deck and final report write up
df["listed_in"] = (
    df["listed_in"]
    .fillna("")                    # Remove NaN
    .str.lower()
    .str.replace("&", "and")       # normalize symbol
    .str.split(",")                # Split multi-label genres
)

df = df.explode("listed_in")
df["listed_in"] = df["listed_in"].str.strip()

# Remove empty / invalid genre entries
df = df[df["listed_in"] != ""]

def get_genre_distribution(indices, df):
    genres = (
        df.loc[indices, "listed_in"]
        .dropna()                                                    # Prevent missing values from distorting counts.
        .astype(str)
    )                                                                # Break genres into individual tokens for fair comparison.
    return genres.value_counts(normalize=True)                       # Normalize so catalog and recommendations are comparable.

catalog_genre_dist = get_genre_distribution(df.index, df)            # Establish baseline genre distribution.
recommendation_genre_dist = get_genre_distribution(rec_indices, df)  # Measure genre exposure in recommendations.

genre_bias_df = pd.concat(
    [catalog_genre_dist, recommendation_genre_dist],
    axis=1,
    keys=["Catalog Distribution", "Recommendation Distribution"]
).fillna(0)

print("Bias Auditing – Genre Exposure Analysis: Catalog vs Recommendation Distribution")
display(genre_bias_df.head(10))
print(
    f"INTERPRETATION:\nDifferences between catalog and recommendation \n"
    f"genre distributions indicate whether certain genres are overexposed, \n"
    f"guiding the need for diversity controls. \n"
)
print("\n")


# ---------------------------------------------------------------------------
# 6d. Bias Auditing – Country Exposure Analysis
# ---------------------------------------------------------------------------
# Purpose: Identify whether recommendations over-represent titles
# from dominant production regions.

# Bias focus: Regional dominance in recommendations

# Clean Country field to remove NaN or blank (" ") countries, found this bug during the deck and final report write up
df["country"] = (
    df["country"]
    .fillna("")
    .str.strip()
)

# Remove empty countries
df = df[df["country"] != ""]

def get_country_distribution(indices, df):
    countries = (
        df.loc[indices, "country"]
        .dropna()                                                        # Prevent missing values from distorting counts.
        .astype(str)
    )                                                                    # Tokenize country metadata for consistent comparison.
    return countries.value_counts(normalize=True)

catalog_country_dist = get_country_distribution(df.index, df)            # Baseline regional distribution.
recommendation_country_dist = get_country_distribution(rec_indices, df)  # Regional exposure in recommendations.

country_bias_df = pd.concat(
    [catalog_country_dist, recommendation_country_dist],
    axis=1,
    keys=["Catalog Distribution", "Recommendation Distribution"]
).fillna(0)

print("Bias Auditing – Country Exposure: Catalog vs Recommendation Distribution")
display(country_bias_df.head(10))
print(
    f"INTERPRETATION:\nThis table highlights whether recommendations \n"
    f"favor titles from specific countries, which may require mitigation \n"
    f"to ensure balanced catalog exposure. \n"
)
print("\n")


# ---------------------------------------------------------------------------
# 6e. Fairness Metrics – Why Demographic Metrics Are Not Applicable
# ---------------------------------------------------------------------------
# Purpose: Clearly justify why standard demographic fairness metrics
# do not apply to this recommender system.

fairness_explanation = pd.DataFrame({
    "Aspect": [
        "User Demographics",
        "Protected Attributes",
        "Prediction Outcomes",
        "Applicable Fairness Definition"
    ],
    "Status": [
        "Not Available",
        "Not Collected",
        "No Predictions Generated",
        "Content Exposure & Diversity"
    ]
})                                                               # Explicitly define fairness scope to avoid misuse.

print("Fairness Assessment Scope and Applicability")
display(fairness_explanation)
print(
    "INTERPRETATION:\nSince the system does not model users or predict outcomes, \n"
    "fairness is evaluated through balanced content exposure rather than \n"
    "demographic parity or similar metrics. \n"
)
print("\n")


# ---------------------------------------------------------------------------
# 6f. Mitigation Strategies (Conceptual + Implemented)
# ---------------------------------------------------------------------------
# Purpose: Map identified risks to concrete actions already implemented
# in the system design.

mitigation_strategies = pd.DataFrame({
    "Bias Risk": [
        "Genre dominance",
        "Country dominance",
        "Duplicate-heavy recommendations",
        "Overly narrow themes"
    ],
    "Mitigation Strategy": [
        "Intra-list Diversity (ILD)",
        "Balanced feature weighting",
        "Exclude near-duplicates",
        "Weighted hybrid similarity"
    ]
})                                                               # Connect risks directly to implemented controls.

print("Identified Bias Risks and Mitigation Strategies")
display(mitigation_strategies)
print(
    "INTERPRETATION:\nThese mitigation strategies demonstrate proactive \n"
    "steps taken to reduce bias and promote fair, diverse recommendations. \n"
)
print("\n")

# Bias Audit Summary Table
# ---------------------------------------------------------------------------
bias_audit_summary = pd.DataFrame({
    "Bias Risk": [
        "Genre dominance",
        "Country dominance",
        "Overly similar recommendations",
        "Narrow thematic exposure"
    ],
    "Detection Metric": [
        "Genre distribution vs catalog baseline",
        "Country distribution vs catalog baseline",
        "Intra-list Diversity (ILD)",
        "Catalog Coverage (CC)"
    ],
    "Mitigation Applied": [
        "Genre-aware similarity weighting",
        "Balanced feature weighting",
        "Intra-list Diversity (ILD)",
        "Weighted hybrid similarity + coverage-aware selection"
    ]
})

print("Bias Audit Summary: Risk, Detection, and Mitigation")
pd.set_option("display.max_colwidth", None)

display(bias_audit_summary.style.set_properties(**{"white-space": "pre-wrap","text-align": "left"}))

print(
    "INTERPRETATION:\n"
    "This summary demonstrates that bias identification, measurement, and mitigation \n"
    "are explicitly linked. Each identified exposure risk is paired with an appropriate \n"
    "proxy metric and a concrete mitigation strategy already implemented in the system, \n"
    "ensuring responsible and auditable recommendation behavior.\n"
)
print("\n")

# ---------------------------------------------------------------------------
# 6g. Limitations & Honest Disclosure
# ---------------------------------------------------------------------------
# Purpose: Transparently document system limitations to set realistic
# expectations for stakeholders.

limitations = pd.DataFrame({
    "Limitation": [
        "Catalog imbalance",
        "Potential data leakage",
        "Generalization to new titles"
    ],
    "Discussion": [
        "Popular genres and countries may still dominate; diversity metrics "
        "and weighting help reduce this effect.",
        "Similarity models use the full catalog; future work could separate "
        "evaluation data for stronger validation.",
        "Performance depends on metadata quality and may vary for newly added "
        "or poorly described titles."
    ]
})                                                               # Explicit disclosure improves trust and auditability.

print("Model Limitations and Responsible Use Considerations")
display(limitations)
print(
    "INTERPRETATION:\nClearly stating limitations strengthens credibility \n"
    "and demonstrates responsible model evaluation. \n"
)

# ---------------------------------------------------------------------------
# 6h. Future Work and System Extensions
# ---------------------------------------------------------------------------
# Purpose: Document realistic and responsible extensions that could enhance
# system performance, evaluation rigor, and business impact beyond the
# current data-limited scope.

print("Future Work and System Extensions")
print("\n")

print(
    "This recommender system was intentionally designed as a content-based solution \n"
    "to operate under data-limited conditions. While effective for cold-start and \n"
    "explainability-driven use cases, several extensions could further improve \n"
    "performance and real-world impact.\n"
)

print(
    "First, incorporating user interaction signals such as viewing history, watch \n"
    "duration, or implicit feedback would enable a hybrid recommendation approach. \n"
    "Combining content similarity with collaborative filtering could improve \n"
    "personalization while retaining explainability for new or low-activity users.\n"
)

print(
    "Second, replacing proxy evaluation metrics with online experimentation methods \n"
    "such as A/B testing would allow the system to be optimized directly against \n"
    "business outcomes. Metrics like click-through rate, completion rate, and session \n"
    "duration would provide stronger evidence of real user engagement.\n"
)

print(
    "Finally, continuous monitoring of catalog growth and metadata quality would help \n"
    "maintain recommendation performance over time. Periodic retraining and repeated \n"
    "bias audits would ensure that the system continues to surface diverse, relevant, \n"
    "and representative content as the catalog evolves.\n"
)


### 𓂃🖊 Key Findings

Explainability analysis shows that recommendations are driven by interpretable content signals, using term overlap as a transparent proxy for model reasoning similar to SHAP or LIME-style local explanations. Shared keywords such as “father,” “life,” “death,” and “documentaries” explain why titles like Dick Johnson Is Dead are surfaced, allowing stakeholders to trace recommendations directly to meaningful semantic features. Feature contribution analysis further confirms a balanced weighting between genre (0.40) and description (0.60), ensuring interpretability while preserving semantic richness and business relevance.

Bias auditing reveals measurable but controlled exposure differences between the catalog and recommendations. For example, documentaries represent approximately 0.043 of the catalog yet account for 0.600 of recommendations, while international documentary titles appear at 0.400 despite comprising only 0.021 of the catalog. At the country level, U.S. titles account for 0.353 of the catalog and 0.200 of recommendations, while U.K. titles increase from 0.053 to 0.800 for specific anchors. These shifts reflect semantic relevance rather than uncontrolled amplification, indicating that recommendations are driven by content similarity rather than popularity bias.

Importantly, diversity controls moderate these effects. Intra-list Diversity remains stable at approximately 0.34, and Catalog Coverage remains near 0.05, indicating that recommendations avoid excessive repetition or collapse toward narrow subsets of content. Together, these results demonstrate that bias is actively monitored and mitigated through explainable feature weighting and diversity-aware selection, supporting transparent, balanced, and responsible recommendation behavior under data-limited conditions.

---

## 🏁 Conclusion

In [ ]:
# =============================================================================
# CONCLUSION
# =============================================================================

print(
    "FINAL CONCLUSION:\n\n"
    "The solution delivers a robust content-based recommender designed for large catalogs \n"
    "and cold-start environments where user interaction data is limited. By balancing \n"
    "relevance, diversity, catalog exposure, and explainability, the system produces \n"
    "stable, interpretable, and business-aligned recommendations rather than optimizing \n"
    "for relevance alone.\n\n"

    "The modeling approach progresses from simple, transparent baselines to semantic \n"
    "similarity methods, ensuring each design choice is justified, measurable, and \n"
    "auditable. Evaluation, explainability, and bias auditing confirm controlled behavior \n"
    "under data-limited conditions and readiness for responsible deployment.\n\n"

    "For Netflix, this approach improves content discovery even when user history is sparse, \n"
    "helping new or infrequent viewers find relevant titles faster. By reducing browsing \n"
    "time and surfacing diverse recommendations, the system supports viewer engagement \n"
    "while avoiding over-reliance on already popular content. The emphasis on explainability \n"
    "and bias-aware evaluation also strengthens trust, governance, and long-term scalability, \n"
    "providing a solid foundation for future hybrid and personalized recommendation systems."
)
